# Représentation graphique

Nous allons parler ici un petit peu de représentation graphique, à partir du jeu de donnée de log apache (que nous pouvons charger en mémoire à l'aide de la cellule suivante) :

In [ ]:
import pandas as pd
import numpy as np
import time

## Le chargement du fichier proprement dit, on indique que les colonnes sont 
## séparées par des ;
apache_log_df = pd.read_csv( "log_aggreges.txt.csv", sep =";" )

## Une fonction pour convertir la colonne "apache_date" en date manipulable par python
apache_log_df["date"] = apache_log_df["apache_date"].apply( 
    lambda apache_date_format: time.strptime(apache_date_format, "%d/%b/%Y:%H:%M:%S") 
     )

## Une fonction pour convertir la colonne "apache_date" en date manipulable par python
apache_log_df["data_size"] = apache_log_df["data_size"].replace("-","0").astype(np.int64)

## Une fonction pour extraire l'heure
apache_log_df["heure"] = apache_log_df["date"].apply( lambda x:x.tm_hour )
apache_log_df["date_heure"] = apache_log_df["date"].apply( lambda x:(x.tm_mday, x.tm_hour ) )

## On peut maintenant avoir un aperçu de ce tableau, avec la fonction head qui permet
## de renvoyer les n premières lignes.
apache_log_df.head(5)

Et nous allons chercher ici à le représenter graphiquement.
La représentation graphique à deux objectifs:

- faciliter la représentation mentale
- faciliter la communication

La librairie utilisée en python pour faire de la représentation graphique est matplotlib.  
A noter qu'elle peut également être utilisée pour faire des animations.  

Vous pouvez trouver de nombreux exemples ici :

http://matplotlib.org/gallery.html

Dans l'ensemble, tracer des graphiques consiste essentiellement à choisir ce que vous voulez mettre en abscisse, ce que vous voulez mettre en ordonnée, et le type de graphique (courbe, histogramme, courbe 3d, surface, etc ...)

Par exemple pour tracer une courbe, vous pourrez utiliser le code suivant :

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(0, 10)
plt.plot(x, np.sin(x), color = 'r', label = "sin(x)")
plt.plot(x, np.cos(x), color = 'g', label = "cos(x)")

plt.legend()
plt.show()

Petite explication, la fonction linspace fournit un étalonnage d'un espace, par défaut 50 points, à partir des bornes. Donc linspace(0,10) fournit 50 points equidistant, compris entre 0 et 10 (les deux bornes étant incluses)

In [ ]:
print( len(np.linspace(0, 10)) )
print( np.linspace(0, 10) )

On utilise la fonction sin de np, plutôt que celle de la librairie standard math, car celle-ci supporte les calculs vectoriels (i.e. on calcule le sinus d'une liste, et on obtient une liste des sinus des éléments). Cela évite de faire des boucles

### Exercice 1

A partir de l'exemple ci-dessus, tracer la courbe de la fonction $ f: x \to x^2 $  
entre -5 et 5.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt



Bien entendu, l'objet n'est pas seulement de tracer des fonctions mathématiques usuelles.  
On peut par exemple tracer la courbe de l'affluence d'un site en fonction de l'heure.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## On récupère le nombre de requête par heure
nb_requete_par_heure = apache_log_df['heure'].value_counts()

plt.plot(nb_requete_par_heure.index, 
         nb_requete_par_heure.values, 
         label = "Nb requete par heure")

plt.legend()
plt.show()

Le résultat n'est pas très satisfaisant, car si les valeurs sont bonnes, l'ordre des valeurs dans l'objet nb_requete_par_heure n'est pas le bon.

On va donc rajouter une étape de tri.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## On récupère le nombre de requête par heure
nb_requete_par_heure = sorted( apache_log_df['heure'].value_counts().items() )

## On trie les heures, et on arrange le 
heures     = [it[0] for it in nb_requete_par_heure]
nb_requete = [it[1] for it in nb_requete_par_heure]


plt.plot(heures, nb_requete, label = "Nb requete par heure")

plt.legend()
plt.show()


## Exercice 2

Faite de même par rapport à la colonne "date_heure", qui contient la date et l'heure.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## On récupère le nombre de requête par heure
nb_requete_par_heure = sorted( apache_log_df['date_heure'].value_counts().items() )

## On trie les heures, et on arrange le 
heures      = [it[0][0]*24 + it[0][1] for it in nb_requete_par_heure]
heures_as_s = ["{0}:{1}".format( it[0][0], it[0][1] ) for it in nb_requete_par_heure]
nb_requete  = [it[1] for it in nb_requete_par_heure]


plt.plot(heures, nb_requete, label = "Nb requete par heure")
plt.xticks(heures, heures_as_s, rotation='vertical')

plt.legend()
plt.show()

## Exercice 3

On va s'intéresser maintenant à un autre type de graphique, les diagrammes, pour les cas où tracer une courbe n'a pas vraiment de sens (par exemple les code de retour).

On peut avoir un résumé des codes de retours ainsi :

In [ ]:
apache_log_df["return_code"].value_counts()

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

return_code_values = apache_log_df["return_code"].value_counts()
n_groups = len(return_code_values)

fig, ax = plt.subplots()

index = np.arange(n_groups)
bar_width = 0.35
rects1 = plt.bar(index + bar_width/2, return_code_values.values, bar_width)

plt.xlabel('Return code')
plt.ylabel('Nombre')
plt.title('Nb return code')
plt.xticks(index + bar_width, return_code_values.index )
plt.legend()

plt.tight_layout()
plt.show()

A vous de jouer !

Essayer maintenant de faire la même chose pour les adresses ip.

## Exercice 4

On va maintenant s'intéresser à un différent type de graphique, les histogrammes.  
Dans ce cas là, on cherche à représenter une distribution, c'est à dire que l'on a pas un ensemble de clé / valeur comme pour les courbes ou les barres, mais un ensemble de valeurs, dont on souhaite analyser la distribution. Dans notre cas, on va commencer par la taille des données de réponse.

In [ ]:
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

num_bins = 50
# the histogram of the data
n, bins, patches = plt.hist(apache_log_df["data_size"], 
                            num_bins, normed=1, 
                            facecolor='green', 
                            alpha=0.5)
# add a 'best fit' line
plt.xlabel('Data size')
plt.ylabel('Probability')
plt.title('Répartition des taille de donnée')

# Tweak spacing to prevent clipping of ylabel
plt.subplots_adjust(left=0.15)
plt.show()



Pas terrible, hein ? De fait, on a un certain nombre de requête dont la taille des données n'est pas disponible, mais surtout beaucoup de requête très grosse comme le témoigne la commande suivante :

In [ ]:
apache_log_df["data_size"].value_counts()

Comme l'histogramme cherche à faire des parties de largeur égale, les très grosses requête provoque une distorsion. On va donc chercher à enlever les 1% de requêtes les plus importantes :

In [ ]:
sorted_data_size = sorted( apache_log_df["data_size"] )
dernier_decile = sorted_data_size[ 9*len(sorted_data_size) // 10 ]
print( dernier_decile )

In [ ]:
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

normed_data_size = apache_log_df["data_size"]
normed_data_size[ apache_log_df["data_size"] > dernier_decile] = dernier_decile

num_bins = 50
# the histogram of the data
n, bins, patches = plt.hist(normed_data_size, 
                            num_bins, normed=1, 
                            facecolor='green', 
                            alpha=0.5)
# add a 'best fit' line
plt.xlabel('Data size')
plt.ylabel('Probability')
plt.title('Répartition des taille de donnée')

# Tweak spacing to prevent clipping of ylabel
plt.subplots_adjust(left=0.15)
plt.show()